In [1]:
import pandas as pd
import numpy as np

CSV_MANHATTAN_INPUT_PATH = r""

x_test_e28 = pd.read_csv(CSV_MANHATTAN_INPUT_PATH +
                         r"\28 distanze\test2.csv", header=None)
y_test_e28 = x_test_e28.iloc[:, 0]
x_test_e28 = x_test_e28.drop(0, axis=1)
y_test_e28 = y_test_e28.to_numpy(np.int32)
x_test_e28 = x_test_e28.to_numpy(np.float32)

print("x_test:", x_test_e28.shape, x_test_e28.dtype)
print("y_test:", y_test_e28.shape, y_test_e28.dtype)


x_test: (36, 6720) float32
y_test: (36,) int32


In [2]:
CSV_IMAGE_INPUT_PATH = r""

x_train = pd.read_csv(CSV_IMAGE_INPUT_PATH + "\\" + "train.csv", header=None)
x_test = pd.read_csv(CSV_IMAGE_INPUT_PATH + "\\" + "test.csv", header=None)
y_train = x_train.iloc[:, 0]
x_train = x_train.drop(0, axis=1)
y_test = x_test.iloc[:, 0]
x_test = x_test.drop(0, axis=1)

y_train = y_train.to_numpy(np.int32)
x_train = x_train.to_numpy(np.float32)
y_test = y_test.to_numpy(np.int32)
x_test = x_test.to_numpy(np.float32)

x_train = x_train.reshape(x_train.shape[0], 100, 150, -1)
x_test = x_test.reshape(x_test.shape[0], 100, 150, -1)

mean_pixel = x_train.mean(axis=(0, 1, 2), keepdims=True)
std_pixel = x_train.std(axis=(0, 1, 2), keepdims=True)
x_test = (x_test - mean_pixel) / std_pixel

print("x_test:", x_test.shape, x_test.dtype)
print("y_test:", y_test.shape, y_test.dtype)


x_test: (72, 100, 150, 3) float32
y_test: (72,) int32


In [3]:
from tensorflow import keras
from joblib import load

MODEL_INPUT_PATH = r""

labels = np.unique(y_train)

model = keras.models.load_model(MODEL_INPUT_PATH + "\\dl\\v1.0")
prediction_dl = model.predict(x_test.reshape(-1, 100, 150, 3))

correct = np.count_nonzero(labels[prediction_dl.argmax(axis=1)] == y_test)
accuracy = correct * 100.0 / y_test.size
print(accuracy)

clf = load(MODEL_INPUT_PATH + "\\ml\\v1.0\\filename.joblib")

prediction = []
temp = []
for i, id in enumerate(y_test):
    temp_dl = model.predict(x_test[i].reshape(-1, 100, 150, 3))[0]
    temp_ml = clf.predict_proba(x_test_e28[np.where(y_test_e28 == id)])[0]
    for j, label in enumerate(labels):
        t = (temp_dl[j] * 0.80) + (temp_ml[int(str(label)[:1]) - 1] * 0.20)
        temp.append(t)
        """ print(temp_dl[j])
        print(temp_ml[int(str(label)[:1]) - 1])
        print(t)
        print("\n") """
    prediction.append(np.array(temp).argmax())
    temp.clear()

correct = np.count_nonzero(labels[prediction] == y_test)
accuracy = correct * 100.0 / y_test.size
print(accuracy)


81.94444444444444
81.94444444444444
